In [1]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [48]:
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config_FY import gKey

In [3]:
l = ['2012', '2013', '2014','2015', '2016']

myList_zip= []
myList_city = []

for item in l:
    
    #dataframe_name = f"census_{item}"
    df = pd.read_csv(f"Data/census_{item}.csv")
    df = df.reset_index(inplace=False, drop=True)
    df['year']=item
    
    #clean census data
    
    df = df.sort_values(by="Zipcode", ascending = True)
    #remove erroraneous values in the census data
    df = df[df["Median Household Income"]>0]
    df = df[df["median_gross_rent"]>0]
    df = df[df["median_monthly_owner_costs"]>0]
    #convert to numeric 
    df = df.apply(pd.to_numeric, errors='ignore')
    
    #add city, state name, lat and lng columns to df
    city_list = []
    state_list = []
    lat_list = []
    lng_list = []
    
    for zipCode in df['Zipcode'].tolist():
        
        #match info to zip code
        city = search.by_zipcode(zipCode).to_dict()['major_city']
        state = search.by_zipcode(zipCode).to_dict()['state']
        lat = search.by_zipcode(zipCode).to_dict()['lat']
        lng = search.by_zipcode(zipCode).to_dict()['lng']
        #collect info into lists
        city_list.append(city)
        state_list.append(state)
        lat_list.append(lat)
        lng_list.append(lng)
       
    #add lists to df    
    df['City'] = city_list
    df['State'] = state_list
    df['Lat'] = lat_list
    df['Lng'] = lng_list
    
    

    #create a df grouped by city, state
    city_state_grp = df.groupby(["State", "City"])

    new_df = pd.DataFrame({f"{item} Median Age": city_state_grp["Median Age"].mean(),
                           f"{item} Total Population": city_state_grp["Total Population"].sum(),
                           f"{item} Poverty Count": city_state_grp["Poverty Count"].sum(),
                           f"{item} Median Household Income": city_state_grp["Median Household Income"].mean(),
                           f"{item} Employed": city_state_grp["employment_employed"].sum(),
                           f"{item} Unemployed": city_state_grp["employment_unemployed"].sum(),
                           f"{item} Monthly Rent": city_state_grp["median_gross_rent"].mean(),
                           f"{item} Owner Monthly Costs": city_state_grp["median_monthly_owner_costs"].mean()
                        })  
      
                    
    #new_df.reset_index()
          
    #obtain list of dfs by city, state name
    myList_city.append(new_df)
    
    #rename dfs
    df =  df.rename(columns={"Median Age": f"{item} Median Age",
                             "Total Population": f"{item} Total Population",
                             "Poverty Count": f"{item} Poverty Count",
                             "Median Household Income": f"{item} Median Household Income",
                             "employment_employed": f"{item} Employed",
                             "employment_unemployed": f"{item} Unemployed",
                             "median_gross_rent": f"{item} Monthly Rent",
                             "median_monthly_owner_costs": f"{item} Owner Monthly Costs"})
    
    df = df[['City', 'State', 'Zipcode', 'Lat', 'Lng',
             f"{item} Median Age", f"{item} Total Population", f"{item} Poverty Count", 
             f"{item} Median Household Income", f"{item} Employed", f"{item} Unemployed",
             f"{item} Monthly Rent", f"{item} Owner Monthly Costs"]]
    
    #obtain list of dfs by zip code
    myList_zip.append(df)

In [39]:
myList_city[3].head()

,State,City,2015 Median Age,2015 Total Population,2015 Poverty Count,2015 Median Household Income,2015 Employed,2015 Unemployed,2015 Monthly Rent,2015 Owner Monthly Costs,"City, State"
0,AK,Akiachak,26.8,562.0,146.0,38750.0,145.0,46.0,725.0,450.0,"Akiachak, AK"
1,AK,Akiak,25.4,399.0,125.0,42000.0,113.0,50.0,775.0,525.0,"Akiak, AK"
2,AK,Akutan,42.9,899.0,145.0,24750.0,864.0,6.0,325.0,400.0,"Akutan, AK"
3,AK,Aleknagik,40.1,199.0,23.0,41875.0,59.0,20.0,413.0,1083.0,"Aleknagik, AK"
4,AK,Ambler,29.5,280.0,60.0,42344.0,98.0,21.0,850.0,1042.0,"Ambler, AK"


In [4]:
for i in range(0, len(myList_city)):
    myList_city[i] = myList_city[i].reset_index()
    myList_city[i]["City, State"] = myList_city[i]['City']+ ", " + myList_city[i]['State']

In [5]:
merge_zip = pd.merge(myList_zip[0], myList_zip[1], how='outer', on='Zipcode' )
merge_city = pd.merge(myList_city[0], myList_city[1], how='outer', on='City, State' )

for i in range(2, len(l)):
    merge_zip = pd.merge(merge_zip, myList_zip[i], how='outer', on='Zipcode')
    merge_city = pd.merge(merge_city, myList_city[i], how='outer', on='City, State')

In [40]:
merge_city.head()

,2012 Median Age,2012 Total Population,2012 Poverty Count,2012 Median Household Income,2012 Employed,2012 Unemployed,2012 Monthly Rent,2012 Owner Monthly Costs,"City, State",2013 Median Age,...,Monthly Rent Growth Rate (2015 to 2016),Owner Monthly Costs Growth Rate (2015 to 2016),5-Year Average Population Growth Rate,5-Year Average Age Growth Rate,5-Year Average Poverty Count Growth Rate,5-Year Average Median Household Income Growth Rate,5-Year Average Employed Growth Rate,5-Year Average Unemployed Growth Rate,5-Year Average Monthly Rent Growth Rate,5-Year Average Owner Monthly Costs Growth Rate
0,43.1,661.0,116.0,27031.0,247.0,4.0,579.0,1018.0,"Granville, WV",43.1,...,0.052464,-0.065073,39.100810,-5.763952,44.988900,4.895119,45.974887,90.673077,3.516147,-7.590063
1,56.1,471.0,27.0,63520.0,203.0,21.0,1375.0,1996.0,"Odessa, DE",55.8,...,-0.167005,-0.007711,30.576577,-8.621733,25.746187,9.512039,31.664623,19.096880,-2.159022,-3.899914
2,47.3,521.0,68.0,39268.0,235.0,66.0,523.0,1163.0,"Williamsport, TN",41.6,...,0.262185,-0.025217,29.568720,-7.301365,85.808816,-4.775454,26.971321,-4.552111,11.638294,1.644352
3,48.2,448.0,81.0,35750.0,185.0,13.0,148.0,918.0,"Seville, FL",43.5,...,-0.035917,-0.007246,27.151084,-3.769747,66.072702,6.842026,31.507831,10.526973,43.102603,1.252009
4,34.9,3880.0,330.0,81290.0,1691.0,112.0,1567.0,2247.0,"Ponte Vedra, FL",36.2,...,0.134442,0.036018,26.292596,0.931991,-9.416824,4.550511,25.439082,27.217690,1.632026,-1.954951


In [6]:

merge_city.drop(['State_x', 'State_y', 'City_x', 'City_y'], axis=1, inplace=True)

In [7]:
merge_zip.drop(['State_x', 'State_y', 'City_x', 'City_y','Lat_x','Lat_y', 'Lng_x', 'Lng_y'], axis=1, inplace=True)

In [8]:
merge_zip.dropna(inplace=True)
merge_city.dropna(inplace=True)

In [20]:
for year in [2013, 2014, 2015, 2016]:
    
    merge_city[f"Population Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Total Population"] - merge_city[f"{year-1} Total Population"])/merge_city[f"{year-1} Total Population"]
    merge_city[f"Median Age Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Median Age"] - merge_city[f"{year-1} Median Age"])/merge_city[f"{year-1} Median Age"]
    merge_city[f"Poverty Count Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Poverty Count"] - merge_city[f"{year-1} Poverty Count"])/merge_city[f"{year-1} Poverty Count"]
    merge_city[f"Median Household Income Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Median Household Income"] - merge_city[f"{year-1} Median Household Income"])/merge_city[f"{year-1} Median Household Income"]
    merge_city[f"Employed Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Employed"] - merge_city[f"{year-1} Employed"])/merge_city[f"{year-1} Employed"]
    merge_city[f"Unemployed Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Unemployed"] - merge_city[f"{year-1} Unemployed"])/merge_city[f"{year-1} Unemployed"]
    merge_city[f"Monthly Rent Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Monthly Rent"] - merge_city[f"{year-1} Monthly Rent"])/merge_city[f"{year-1} Monthly Rent"]
    merge_city[f"Owner Monthly Costs Growth Rate ({year-1} to {year})"] = (merge_city[f"{year} Owner Monthly Costs"] - merge_city[f"{year-1} Owner Monthly Costs"])/merge_city[f"{year-1} Owner Monthly Costs"]
    
    
    #merge_zip[f"Population Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Total Population"] - merge_zip[f"{year-1} Total Population"])/merge_zip[f"{year-1} Total Population"]
    #merge_zip[f"Median Age Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Median Age"] - merge_zip[f"{year-1} Median Age"])/merge_zip[f"{year-1} Median Age"]
    #merge_zip[f"Poverty Count Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Poverty Count"] - merge_zip[f"{year-1} Poverty Count"])/merge_zip[f"{year-1} Poverty Count"]
    #merge_zip[f"Median Household Income Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Median Household Income"] - merge_zip[f"{year-1} Median Household Income"])/merge_zip[f"{year-1} Median Household Income"]
    #merge_zip[f"Employed Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Employed"] - merge_zip[f"{year-1} Employed"])/merge_zip[f"{year-1} Employed"]
    #merge_zip[f"UnemplGrowth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Unemployed"] - merge_zip[f"{year-1} Unemployed"])/merge_zip[f"{year-1} Unemployed"]
    #merge_zip[f"Month Rent Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Monthly Rent"] - merge_zip[f"{year-1} Monthly Rent"])/merge_zip[f"{year-1} Monthly Rent"]
    #merge_zip[f"Owner Monthly Costs Growth Rate ({year-1} to {year})"] = (merge_zip[f"{year} Owner Monthly Costs"] - merge_zip[f"{year-1} Owner Monthly Costs"])/merge_zip[f"{year-1} Owner Monthly Costs"]
    
merge_city["5-Year Average Population Growth Rate"] = 100*sum([merge_city["Population Growth Rate (2012 to 2013)"],
                                                           merge_city["Population Growth Rate (2013 to 2014)"],
                                                           merge_city["Population Growth Rate (2014 to 2015)"],
                                                           merge_city["Population Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Age Growth Rate"] = 100*sum([merge_city["Median Age Growth Rate (2012 to 2013)"],
                                                    merge_city["Median Age Growth Rate (2013 to 2014)"],
                                                    merge_city["Median Age Growth Rate (2014 to 2015)"],
                                                    merge_city["Median Age Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Poverty Count Growth Rate"] = 100*sum([merge_city["Poverty Count Growth Rate (2012 to 2013)"],
                                                              merge_city["Poverty Count Growth Rate (2013 to 2014)"],
                                                              merge_city["Poverty Count Growth Rate (2014 to 2015)"],
                                                              merge_city["Poverty Count Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Median Household Income Growth Rate"] = 100*sum([merge_city["Median Household Income Growth Rate (2012 to 2013)"],
                                                                        merge_city["Median Household Income Growth Rate (2013 to 2014)"],
                                                                        merge_city["Median Household Income Growth Rate (2014 to 2015)"],
                                                                        merge_city["Median Household Income Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Employed Growth Rate"] = 100*sum([merge_city["Employed Growth Rate (2012 to 2013)"],
                                                         merge_city["Employed Growth Rate (2013 to 2014)"],
                                                         merge_city["Employed Growth Rate (2014 to 2015)"],
                                                         merge_city["Employed Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Unemployed Growth Rate"] = 100*sum([merge_city["Unemployed Growth Rate (2012 to 2013)"],
                                                           merge_city["Unemployed Growth Rate (2013 to 2014)"],
                                                           merge_city["Unemployed Growth Rate (2014 to 2015)"],
                                                           merge_city["Unemployed Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Monthly Rent Growth Rate"] = 100*sum([merge_city["Monthly Rent Growth Rate (2012 to 2013)"],
                                                             merge_city["Monthly Rent Growth Rate (2013 to 2014)"],
                                                             merge_city["Monthly Rent Growth Rate (2014 to 2015)"],
                                                             merge_city["Monthly Rent Growth Rate (2015 to 2016)"]])/4
merge_city["5-Year Average Owner Monthly Costs Growth Rate"] = 100*sum([merge_city["Owner Monthly Costs Growth Rate (2012 to 2013)"],
                                                                    merge_city["Owner Monthly Costs Growth Rate (2013 to 2014)"],
                                                                    merge_city["Owner Monthly Costs Growth Rate (2014 to 2015)"],
                                                                    merge_city["Owner Monthly Costs Growth Rate (2015 to 2016)"]])/4

In [21]:
merge_city.sort_values(by="5-Year Average Population Growth Rate", inplace=True, ascending=False)
merge_city.reset_index(drop=True, inplace=True)
merge_city.to_csv("merge_Growth_by_City.csv")


In [23]:
merge_city.head()

,2012 Median Age,2012 Total Population,2012 Poverty Count,2012 Median Household Income,2012 Employed,2012 Unemployed,2012 Monthly Rent,2012 Owner Monthly Costs,"City, State",2013 Median Age,...,Monthly Rent Growth Rate (2015 to 2016),Owner Monthly Costs Growth Rate (2015 to 2016),5-Year Average Population Growth Rate,5-Year Average Age Growth Rate,5-Year Average Poverty Count Growth Rate,5-Year Average Median Household Income Growth Rate,5-Year Average Employed Growth Rate,5-Year Average Unemployed Growth Rate,5-Year Average Monthly Rent Growth Rate,5-Year Average Owner Monthly Costs Growth Rate
0,43.1,661.0,116.0,27031.0,247.0,4.0,579.0,1018.0,"Granville, WV",43.1,...,0.052464,-0.065073,39.100810,-5.763952,44.988900,4.895119,45.974887,90.673077,3.516147,-7.590063
1,56.1,471.0,27.0,63520.0,203.0,21.0,1375.0,1996.0,"Odessa, DE",55.8,...,-0.167005,-0.007711,30.576577,-8.621733,25.746187,9.512039,31.664623,19.096880,-2.159022,-3.899914
2,47.3,521.0,68.0,39268.0,235.0,66.0,523.0,1163.0,"Williamsport, TN",41.6,...,0.262185,-0.025217,29.568720,-7.301365,85.808816,-4.775454,26.971321,-4.552111,11.638294,1.644352
3,48.2,448.0,81.0,35750.0,185.0,13.0,148.0,918.0,"Seville, FL",43.5,...,-0.035917,-0.007246,27.151084,-3.769747,66.072702,6.842026,31.507831,10.526973,43.102603,1.252009
4,34.9,3880.0,330.0,81290.0,1691.0,112.0,1567.0,2247.0,"Ponte Vedra, FL",36.2,...,0.134442,0.036018,26.292596,0.931991,-9.416824,4.550511,25.439082,27.217690,1.632026,-1.954951


In [24]:
merge_city.columns

Index(['2012 Median Age', '2012 Total Population', '2012 Poverty Count',
       '2012 Median Household Income', '2012 Employed', '2012 Unemployed',
       '2012 Monthly Rent', '2012 Owner Monthly Costs', 'City, State',
       '2013 Median Age', '2013 Total Population', '2013 Poverty Count',
       '2013 Median Household Income', '2013 Employed', '2013 Unemployed',
       '2013 Monthly Rent', '2013 Owner Monthly Costs', '2014 Median Age',
       '2014 Total Population', '2014 Poverty Count',
       '2014 Median Household Income', '2014 Employed', '2014 Unemployed',
       '2014 Monthly Rent', '2014 Owner Monthly Costs', '2015 Median Age',
       '2015 Total Population', '2015 Poverty Count',
       '2015 Median Household Income', '2015 Employed', '2015 Unemployed',
       '2015 Monthly Rent', '2015 Owner Monthly Costs', 'State', 'City',
       '2016 Median Age', '2016 Total Population', '2016 Poverty Count',
       '2016 Median Household Income', '2016 Employed', '2016 Unemployed',
    

In [46]:
merge_city[merge_city['2012 Total Population']>= 200000].to_csv("merge_city_pop_greater_than_200k.csv")

In [25]:
#select columns needed
merge_city_5yAverage = merge_city[['City', 'State', '2012 Total Population',
                                  '5-Year Average Population Growth Rate',
                                  '5-Year Average Age Growth Rate',
                                  '5-Year Average Poverty Count Growth Rate',
                                  '5-Year Average Median Household Income Growth Rate',
                                  '5-Year Average Employed Growth Rate',
                                  '5-Year Average Unemployed Growth Rate',
                                  '5-Year Average Monthly Rent Growth Rate',
                                  '5-Year Average Owner Monthly Costs Growth Rate']]


#select cities with 2012 base population >= 200,000
merge_city_5yAverage = merge_city_5yAverage[merge_city_5yAverage['2012 Total Population']>=200000]

merge_city_5yAverage.head()

,City,State,2012 Total Population,5-Year Average Population Growth Rate,5-Year Average Age Growth Rate,5-Year Average Poverty Count Growth Rate,5-Year Average Median Household Income Growth Rate,5-Year Average Employed Growth Rate,5-Year Average Unemployed Growth Rate,5-Year Average Monthly Rent Growth Rate,5-Year Average Owner Monthly Costs Growth Rate
1003,Katy,TX,247294.0,5.924817,0.632505,2.458720,1.698790,6.507591,2.387433,2.489258,0.071222
2252,Irvine,CA,214291.0,3.699406,0.376676,6.953642,0.198074,3.058651,0.897965,2.459216,-1.015132
2383,Kissimmee,FL,232075.0,3.552188,0.383692,7.990001,-0.120192,2.888022,-6.789064,0.613190,-3.393180
2939,Spring,TX,323197.0,3.010138,0.493035,2.651771,1.505416,2.607840,-4.213583,3.375424,0.448610
3474,New Orleans,LA,379926.0,2.606485,0.556808,1.782645,1.853025,3.083344,-1.498844,1.206043,0.413636


In [26]:
merge_city_5yAverage["Lat"] = ""
merge_city_5yAverage["Lng"] = ""


# Geocode the cities
params = {"key": gKey}

# Loop through the merge_city_5yAverage and run a lat/long search for each city
for index, row in merge_city_5yAverage.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    city =  row['City']
    state =  row['State']
    
    # update address key value
    params["address"] = f"{city},{state}"
    
    # make requests
    cities_lat_lng = requests.get(base_url, params=params).json()
    
    merge_city_5yAverage.loc[index, "Lat"] = float(cities_lat_lng["results"][0]["geometry"]["location"]["lat"])
    merge_city_5yAverage.loc[index, "Lng"] = float(cities_lat_lng["results"][0]["geometry"]["location"]["lng"])

merge_city_5yAverage.head(10)
    
    

,City,State,2012 Total Population,5-Year Average Population Growth Rate,5-Year Average Age Growth Rate,5-Year Average Poverty Count Growth Rate,5-Year Average Median Household Income Growth Rate,5-Year Average Employed Growth Rate,5-Year Average Unemployed Growth Rate,5-Year Average Monthly Rent Growth Rate,5-Year Average Owner Monthly Costs Growth Rate,Lat,Lng
1003,Katy,TX,247294.0,5.924817,0.632505,2.458720,1.698790,6.507591,2.387433,2.489258,0.071222,29.7858,-95.8244
2252,Irvine,CA,214291.0,3.699406,0.376676,6.953642,0.198074,3.058651,0.897965,2.459216,-1.015132,33.6846,-117.827
2383,Kissimmee,FL,232075.0,3.552188,0.383692,7.990001,-0.120192,2.888022,-6.789064,0.613190,-3.393180,28.292,-81.4076
2939,Spring,TX,323197.0,3.010138,0.493035,2.651771,1.505416,2.607840,-4.213583,3.375424,0.448610,30.0799,-95.4172
3474,New Orleans,LA,379926.0,2.606485,0.556808,1.782645,1.853025,3.083344,-1.498844,1.206043,0.413636,29.9511,-90.0715


In [45]:
merge_city_5yAverage.head(10)

,City,State,2012 Total Population,5-Year Average Population Growth Rate,5-Year Average Age Growth Rate,5-Year Average Poverty Count Growth Rate,5-Year Average Median Household Income Growth Rate,5-Year Average Employed Growth Rate,5-Year Average Unemployed Growth Rate,5-Year Average Monthly Rent Growth Rate,5-Year Average Owner Monthly Costs Growth Rate,Lat,Lng,text
1003,Katy,TX,247294.0,5.924817,0.632505,2.458720,1.698790,6.507591,2.387433,2.489258,0.071222,29.7858,-95.8244,"Katy, TX Pop Growth Rate: 5.9"
2252,Irvine,CA,214291.0,3.699406,0.376676,6.953642,0.198074,3.058651,0.897965,2.459216,-1.015132,33.6846,-117.827,"Irvine, CA Pop Growth Rate: 3.7"
2383,Kissimmee,FL,232075.0,3.552188,0.383692,7.990001,-0.120192,2.888022,-6.789064,0.613190,-3.393180,28.292,-81.4076,"Kissimmee, FL Pop Growth Rate: 3.6"
2939,Spring,TX,323197.0,3.010138,0.493035,2.651771,1.505416,2.607840,-4.213583,3.375424,0.448610,30.0799,-95.4172,"Spring, TX Pop Growth Rate: 3.0"
3474,New Orleans,LA,379926.0,2.606485,0.556808,1.782645,1.853025,3.083344,-1.498844,1.206043,0.413636,29.9511,-90.0715,"New Orleans, LA Pop Growth Rate: 2.6"
3684,Jamaica,NY,221013.0,2.459357,-0.216112,1.467056,1.118183,2.840704,-0.515700,1.792983,0.470125,40.7027,-73.789,"Jamaica, NY Pop Growth Rate: 2.5"
3752,Austin,TX,954661.0,2.400148,1.093169,-1.137439,2.925616,3.034741,-5.996052,3.842189,1.435023,30.2672,-97.7431,"Austin, TX Pop Growth Rate: 2.4"
4029,Bradenton,FL,219598.0,2.224023,0.848053,3.973129,0.593417,1.961801,-9.689310,3.107354,-4.007652,27.4989,-82.5748,"Bradenton, FL Pop Growth Rate: 2.2"
4178,Charlotte,NC,798534.0,2.143310,0.439466,1.985717,2.134120,2.752327,-2.890301,3.028066,-1.238177,35.2271,-80.8431,"Charlotte, NC Pop Growth Rate: 2.1"
4206,Alexandria,VA,318283.0,2.126363,-0.042717,6.480450,1.751767,2.006348,-0.566090,3.211579,-0.118388,38.8048,-77.0469,"Alexandria, VA Pop Growth Rate: 2.1"


In [42]:
import plotly.plotly as py

In [44]:
df = merge_city_5yAverage

df['text'] = df['City'] + ', ' + df['State'] + ' ' + 'Pop Growth Rate: ' + round(df['5-Year Average Population Growth Rate'],1).astype(str)

#scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    #[0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]
scl = 'custom-colorscale'

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['Lng'],
        lat = df['Lat'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = df['5-Year Average Population Growth Rate'],
            cmax = df['5-Year Average Population Growth Rate'].max(),
            colorbar=dict(
                title="Average Population Growth Rate (%)"
            )
        ))]

layout = dict(
        title = 'U.S. Cities 2012-2016 Average Population Growth Rate (%)',
        colorbar = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-airports' )